In [ ]:
# add root folder of the project to path
import sys
sys.path.insert(0, '../..')

# Load landamarks

In [ ]:
from mesh4d import utils
m8 = utils.load_pkl_object('../../data/landmarks/6kmh_braless_18markers_12fps.pkl')
m8.interp_field()

In [ ]:
from mesh4d.analyse import visual

points_frame1 = m8.get_frame_coord(0).get_points_coord()
pvpcd_frame1 = visual.np2pvpcd(points_frame1, radius=10)

points_frame2 = m8.get_frame_coord(1).get_points_coord()
pvpcd_frame2 = visual.np2pvpcd(points_frame2, radius=10)

Visualise the 1st frame and the 2nd frames as gold balls and green balls, respectively.

In [ ]:
import pyvista as pv

scene = pv.Plotter()
scene.add_mesh(pvpcd_frame1, color='gold')
scene.add_mesh(pvpcd_frame2, color='green')
scene.show()

# Construct the Thin-Plate-Spline motion model

In [ ]:
from scipy.interpolate import RBFInterpolator

disp = points_frame2 - points_frame1
field = RBFInterpolator(points_frame1, disp)

# TPS motion model visualisation

In [ ]:
# movements of the control points
disp_crontrol = field(points_frame1)

In [ ]:
# reconstruct a surface based on all points from the 1st & 2nd frames 
# the points on this surface is used to visualise the displacement of different place
import numpy as np
points = np.concatenate((points_frame1, points_frame2))

points = pv.wrap(points)
surf = points.reconstruct_surface()
points_surf = np.array(surf.points)

In [ ]:
# movements of the surface points
disp_surf = field(points_surf)

In [ ]:
scene = pv.Plotter()

scene.add_mesh(pvpcd_frame1, color='gold', opacity=0.5)
scene.add_mesh(pvpcd_frame2, color='green', opacity=0.5)

for idx in range(len(disp_crontrol)):
    arrow = pv.Arrow(start=points_frame1[idx], direction=disp_crontrol[idx], scale=np.linalg.norm(disp_crontrol[idx]))
    scene.add_mesh(arrow, color='black')

for idx in range(len(disp_surf)):
    arrow = pv.Arrow(start=points_surf[idx], direction=disp_surf[idx], scale=np.linalg.norm(disp_surf[idx]))
    scene.add_mesh(arrow, color='white')

scene.show()